# BALTO Graphical User Interface (prototype)

This Jupyter notebook creates a GUI (graphical user interface) for BALTO (Brokered Alignment of Long-Tail Observations) project.  BALTO is funded by the NSF EarthCube program.  The GUI intends to provide an simplified and customizable method for users to access data sets of interest on servers that support the OpenDAP data access protocol.  The interactive GUI runs within the Jupyter notebook and uses the Python packages: <b>ipywidgets</b> and <b>ipyleaflet</b>.

## Set up a conda environment called "balto"

To run this Jupyter notebook, it is recommended to use Python 3.7 from an Anaconda distribution and to create a "balto" conda environment with the following commands.  

conda update -n base -c defaults conda <br>
conda create --name balto  <br>
conda activate balto  <br>
conda install -c conda-forge ipywidgets <br>
conda install -c conda-forge ipyleaflet <br>
conda install -c conda-forge pydap <br>

conda install -c conda-forge jupyterlab <br>
conda install -c conda-forge nb_conda_kernels  # (needed for conda envs) <br>

conda install -c conda-forge nodejs <br>
conda install -c conda-forge widgetsnbextension <br>
jupyter labextension install jupyter-leaflet <br>
jupyter labextension install @jupyter-widgets/jupyterlab-manager <br>

Change to directory with BALTO_GUI.ipynb. <br>
jupyter lab <br>

Finally, choose BALTO_GUI.ipynb in Jupyter Lab,
but make sure to choose the kernel:  Python [conda evn:balto] <br>

References <br>
https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html <br>
https://ipywidgets.readthedocs.io/en/latest/user_install.html#installing-the-jupyterlab-extension <br>

## Import required packages

In [1]:
from ipyleaflet import Map
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display, HTML
## from IPython.core.display import display
## from IPython.lib.display import display
import pydap

## Create the GUI components

This GUI is built up from <b>ipywidgets</b> (for the controls) and <b>ipyleaflet</b> (for the interactive map).

For more information on ipywidgets, see:  https://ipywidgets.readthedocs.io/en/latest/user_guide.html

For more information on ipyleaflet, see:
https://ipyleaflet.readthedocs.io/en/latest/


In [2]:
p0 = widgets.HTML(value=f"<p></p> <p></p>")   # padding
h0 = widgets.HTML(value=f"<b><font size=5>BALTO User Interface</font></b>")
# p0 = widgets.HTML(value="<p></p> <p></p>")   # padding
# h0 = widgets.HTML(value="<b><font size=5>BALTO User Interface</font></b>")
#---------------------------------------------
# h0 = widgets.Label('BALTO User Interface')

# Create an interactive map with ipyleaflets
map_width     = '600px'
map_height    = '250px'
url_box_width = '560px'
m = Map(center=(0.0, 0.0), zoom=1, 
        layout=Layout(width=map_width, height=map_height))

style0  = {'description_width': 'initial'}
style1  = {'description_width': '130px'}
style2  = {'description_width': '80px'}
style3  = {'description_width': '50px'}
# bbox_style = {'description_width': '130px'}
bbox_style = {'description_width': '100px'}
date_style = {'description_width': '70px'}

#####################################################
# Does "step=001" restrict accuracy of selection ??
#####################################################
bbox_width = '270px'
w1 = widgets.BoundedFloatText(
    value=-180, min=-180, max=180.0, step=0.01,
    # description='West longitude:',
    description='West edge lon:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w2 = widgets.BoundedFloatText(
    value=180, min=-180, max=180.0, step=0.01,
    # description='East longitude:',
    description='East edge lon:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w3 = widgets.BoundedFloatText(
    value=90, min=-90, max=90.0, step=0.01,
    # description='North latitude:',
    description='North edge lat:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
w4 = widgets.BoundedFloatText(
    value=-90, min=-90, max=90.0, step=0.01,
    # description='South latitude:',
    description='South edge lat:',
    disabled=False, style=bbox_style,
    layout=Layout(width=bbox_width) )
    
# Date Range  (& Temporal Resolution ??)
date_width = '270px'
d1 = widgets.DatePicker( description='Start Date:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d2 = widgets.DatePicker( description='End Date:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d3 = widgets.Text( description='Start Time:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )
d4 = widgets.Text( description='End Time:',
            disabled=False, style=date_style,
            layout=Layout(width=date_width) )

# Variable Options
n0 = widgets.HTML(value=f"<p></p>")   # padding
n1 = widgets.Text(description='Variable name:',
                  value='sea surface temperature',
                  disabled=False, style=style0,
                  layout=Layout(width='550px') )             

#------------------------------
# Example GES DISC opendap URL
#------------------------------
# https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHE.05/2014/
# 091/3B-HHR-E.MS.MRG.3IMERG.20140401-S000000-E002959.0000.V05B.HDF5.nc
# ?HQprecipitation[1999:2200][919:1049],lon[1999:2200],lat[919:1049]

# OpenDAP Options
o1 = widgets.Text(description='OpenDAP URL:',
                  value='https://gpm1.gesdisc.eosdis.nasa.gov/opendap/',
                  disabled=False, style=style1,
                  layout=Layout(width=url_box_width))
o2 = widgets.Dropdown( description='OpenDAP package:',
                       options=['pydap', 'netcdf4'],
                       value='pydap',
                       disabled=False, style=style1)

# Output File Format
f1 = widgets.Dropdown( description='Output Format:',
                       options=['HDF', 'netCDF', 'netCDF4'],
                       value='netCDF',
                       disabled=False, style=style0)

## layout=Layout(width=map_width, height=map_height))

# Buttons at the bottom
b1 = widgets.Button(description="Download")

# Can use this for output
status = widgets.Text(description=' Status:', style=style3,
                      layout=Layout(width='380px') )

log = widgets.Textarea( description='', value='',
              disabled=False, style=style0,
              layout=Layout(width='560px', height='160px'))

## Define some GUI utility functions

In [3]:
def get_bounds():
    return [m.west, m.south, m.east, m.north]

#==================================================================
def get_start_date():
    
    if (d1.value is not None):
        return str(d1.value)    # Need the str().
    else:
        return 'None'

#==================================================================
def get_end_date():

    if (d2.value is not None):
        return str(d2.value)   # Need the str().
    else:
        return 'None'

#==================================================================
def get_variable_name():
    return n1.value

#==================================================================
def get_opendap_package():
    return o2.value

#==================================================================
def get_output_format():
    return f1.value

#==================================================================
def list_to_string( array ):

    s = ''
    for item in array:
        s = s + item + '\n'
    return s

#==================================================================   
def print_choices():

    date1, date2 = get_date_range()
    msg = [
    'bounds = ' + str(get_bounds()),
    'opendap package = ' + get_opendap_package(),
    'start date = ' + get_start_date(),
    'end date = ' + get_end_date(),
    'variable = ' + get_variable_name() ]
    log.value = list_to_string( msg )

#==================================================================


## Define some GUI event handling functions

In [4]:
def download_data( b ):
    status.value = 'Download button clicked.'
    print_choices()
    
#==================================================================
def show_bounds( **kwargs ):
    event = kwargs.get('type')
    # events: mouseup, mousedown, mousemove, mouseover, mouseout,
    #         click, dblclick, preclick
    if (event == 'mouseup') or (event == 'mousemove') or (event == 'dblclick'):
        w1.value = m.west
        w2.value = m.east
        w3.value = m.north
        w4.value = m.south
        
    # status.value = event
    
    # with output2:
    #   print( event )
#==================================================================



## Set up the GUI event handlers

In [5]:
b1.on_click( download_data )
m.on_interaction( show_bounds )

## Create the GUI from the GUI components

In [6]:
#===========================
# Set up the UI layout: V1
#===========================
# v1 = widgets.VBox([w1, w2, w3, w4])
# v2 = widgets.VBox([d1, d2, n0, n1 ])
# h1 = widgets.HBox([v1, v2])
# h2 = widgets.VBox([o1, o2]) 
# h3 = widgets.HBox([b1, status])
# ui = widgets.VBox([p0, h0, m, h1, h2, h3, p0, log])

#======================================
# Set up the UI layout: V2: Accordion
#======================================
v1a = widgets.VBox([w1, w2])
v1b = widgets.VBox([w3, w4])
v1  = widgets.HBox( [v1a, v1b])
# v2  = widgets.VBox([d1, d2])
v2a  = widgets.VBox([d1, d2])
v2b  = widgets.VBox([d3, d4])
h2  = widgets.HBox([v2a, v2b])
v3  = widgets.VBox([n1])
v4  = widgets.VBox([o1,o2])
v5  = widgets.VBox([f1])
h3  = widgets.HBox([b1, status])  ####
v6  = widgets.VBox([h3, log])
# selected_index=None causes all cells to be collapsed
acc = widgets.Accordion( children=[v1, h2, v3, v4, v5, v6],
                         selected_index=None,
                         layout=Layout(width=map_width) )
acc.set_title(0,'Spatial Extent')
acc.set_title(1,'Date Range')
acc.set_title(2,'Variable')
acc.set_title(3,'OpenDAP Server')
acc.set_title(4,'Output Format')
acc.set_title(5,'Download')

ui = widgets.VBox([p0,h0,m,acc])

## Display the GUI

In [7]:
gui_output = widgets.Output()
display(ui, gui_output)

Output()

## Some information for testing

In [8]:
# Geographic bounding box for state of Colorado
# Colorado_xmin = -109.060253
# Colorado_xmax = -102.041524
# Colorado_ymin = 36.992426
# Colorado_ymax = 41.003444